In [55]:
%load_ext autoreload
%autoreload 2

import numpy as np

np.set_printoptions(precision=3)
np.random.seed(317042)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
# error reporting function
def create_error_dict(**kwargs):
	error_dict = {}
	
	for err_name, errs in kwargs.items():
		mean = np.mean(errs)
		max = np.max(errs)

		error_dict[f"{err_name}_mean"] = mean
		error_dict[f"{err_name}_max"] = max

	return error_dict

### Stability (and correctness) of Hessenberg-ification

In [57]:
from scipy.linalg import hessenberg
from utils import get_random_symmetric_matrix, make_upper_hessenberg

def test_hessenberg(test_count = 100, size = 5, a = 0.0, b = 1.0):
	As = []
	errors = []

	for t in range(test_count):
		A = get_random_symmetric_matrix(size, a, b)
		As.append(A)

		Hp, _ = make_upper_hessenberg(A)
		H = hessenberg(A)

		errors.append(np.abs(Hp - H))

	return As, errors

In [58]:
As, errs = test_hessenberg(100, 200)
create_error_dict(mae=errs)

{'mae_mean': np.float64(2.7518716752176167e-15),
 'mae_max': np.float64(5.764001498320681e-10)}

### Correctness of Hessenberg QR

In [59]:
from utils import hessenberg_qr

def test_hessenberg_qr(test_count = 100, size = 5, a = 0.0, b = 1.0):
	As = []
	recon_errors = []
	q_errors = []
	r_errors = []

	for t in range(test_count):
		A = get_random_symmetric_matrix(size, a, b)
		As.append(A)

		H, _ = make_upper_hessenberg(A)
		Qp, Rp = hessenberg_qr(H)
		Q, R = np.linalg.qr(H)

		recon_errors.append(np.abs(Qp @ Rp - H))
		q_errors.append(np.abs(Q - Qp))
		r_errors.append(np.abs(R - Rp))

	return As, recon_errors, q_errors, r_errors


In [60]:
As, recon_errors, q_errors, r_errors = test_hessenberg_qr(100, 300)
create_error_dict(reconstruction=recon_errors, q_mae=q_errors, r_mae=r_errors)

{'reconstruction_mean': np.float64(7.595101782397772e-17),
 'reconstruction_max': np.float64(1.1013412404281553e-13),
 'q_mae_mean': np.float64(1.6017246532616314e-16),
 'q_mae_max': np.float64(1.5612323139899413e-13),
 'r_mae_mean': np.float64(2.2490770366452077e-16),
 'r_mae_max': np.float64(1.6631140908884845e-13)}

### Practical-QR Step

In [61]:
from eigen import practical_qr

In [62]:
def test_qr_iteration(test_count = 100, size = 5, a = 0.0, b = 1.0):
	errors = []
	
	for t in range(test_count):
		A = get_random_symmetric_matrix(size, a, b)

		evals, evecs = practical_qr(A)

		err = 0.0
		for j, eval in enumerate(evals):
			vec = evecs[:, j]

			err += np.mean(np.abs(A @ vec - eval * vec))
		
		errors.append(err / len(evals))

	return errors

In [63]:
errs = test_qr_iteration(100, 50)
create_error_dict(evec=errs)

{'evec_mean': np.float64(1.3220890526559452e-11),
 'evec_max': np.float64(2.72293335417927e-11)}

### SVD

In [64]:
from svd import svd

In [65]:
def test_svd(test_count = 100, size = 5, a = 0.0, b = 1.0, square=True):
	errors = []
	
	for t in range(test_count):
		m = size
		n = size

		if not square:
			n = np.random.randint(1, size+1)

			if np.random.uniform() <= 0.5:
				m, n = n, m

		A = np.random.uniform(a, b, (m, n))

		U, S, Vt = svd(A)

		err = 0.0
		for j, sval in enumerate(S):
			v = Vt[j]
			u = U[:, j]

			err += np.mean(np.abs(A @ v - sval * u))
		
		errors.append(err / len(S))

	return errors

In [66]:
create_error_dict(recon_err=test_svd(100, 100, square=True))

{'recon_err_mean': np.float64(2.5495269946592034e-18),
 'recon_err_max': np.float64(2.808740924304527e-18)}

In [67]:
create_error_dict(recon_err=test_svd(100, 100, square=False))

{'recon_err_mean': np.float64(3.612848560453751e-12),
 'recon_err_max': np.float64(7.446476768535604e-11)}